# Пособие по работе с **rag** системой

## Мы рассмотрим:

* Авторизацию в Yandex Cloud
* Работу с файлами: загрузка, выгрузка, хранение
* Поиск по данным (векторный/семантический)
* Интеграцию с YandexGPT для контекстного ответа (RAG)

## Что такое RAG ?

**RAG** (Retrieval-Augmented Generation) — это архитектура, где:

1. Сначала из внешнего источника (базы знаний, файлов, БД) извлекаются релевантные фрагменты данных (retrieval).
2. Затем эти фрагменты подставляются в промпт языковой модели (например, YandexGPT), чтобы она сгенерировала ответ на основе контекста.

Это позволяет получать точные, актуальные и основанные на ваших данных ответы, даже если модель сама не знает этих данных.

## 1. Авторизация в Yandex Cloud

### Для работы с API Yandex Cloud необходимо:

**1. Создать сервисный аккаунт**
* Зайдите в Yandex Cloud Console
* Перейдите в раздел IAM → Сервисные аккаунты
* Создайте новый сервисный аккаунт, например: rag-service-account
* Назначьте ему роли:
    * ai.languageModels.user — для вызова YandexGPT
    * storage.editor — для работы с Object Storage
    * serverless.functions.invoker — если используете функции
    * (опционально) ai.foundationModels.user — для эмбеддингов

**2. Создать статический ключ доступа**
* В консоли: выберите созданный сервисный аккаунт → Авторизационные ключи → Создать новый ключ
* Выберите API-ключ или OAuth-токен (рекомендуется API-ключ для автоматизации)
* Сохраните key_id и secret — они понадобятся для SDK

**3. Установите Yandex Cloud SDK**
>pip install yandexcloud

**4. Авторизация в коде (Python)**

In [ ]:
from yandex.cloud.iam.v1 import iam_token_service_pb2, iam_token_service_pb2_grpc
from yandex.cloud.ai.foundation_models.v1 import text_embedding_service_pb2_grpc, text_embedding_service_pb2
from yandex.cloud.ai.stt.v3 import stt_service_pb2_grpc, stt_service_pb2
import grpc
import os

# Загрузка ключей из .env
from dotenv import load_dotenv
load_dotenv()

YC_API_KEY = os.getenv("YC_API_KEY")

def get_iam_token(api_key):
    channel = grpc.secure_channel('iam.api.cloud.yandex.net:443', grpc.ssl_channel_credentials())
    stub = iam_token_service_pb2_grpc.IamTokenServiceStub(channel)
    request = iam_token_service_pb2.CreateIamTokenRequest(yandex_passport_oauth_token=api_key)
    response = stub.Create(request)
    return response.iam_token

IAM_TOKEN = get_iam_token(YC_API_KEY)

## 2. Работа с файлами: Object Storage

Yandex Cloud предоставляет **Object Storage** — аналог S3 для хранения файлов.

**1. Создайте бакет**
* В консоли: Object Storage → Создать бакет
* Имя: rag-documents-bucket
* Тип: Холодное хранилище (если редкий доступ) или Стандартное
* ACL: Публичный доступ — нет

**2. Установите библиотеку**
>pip install boto3

**3. Загрузка файла в бакет**

In [ ]:
import boto3
from botocore.config import Config

def get_s3_client(iam_token, folder_id):
    config = Config(signature_version='s3v4')
    s3 = boto3.client(
        's3',
        endpoint_url='https://storage.yandexcloud.net',
        aws_access_key_id=iam_token,
        aws_secret_access_key='',
        region_name='ru-central1',
        config=config
    )
    return s3

# Пример загрузки
s3 = get_s3_client(IAM_TOKEN, FOLDER_ID)
s3.upload_file('local_file.txt', 'rag-documents-bucket', 'remote/path/file.txt')

**4. Скачивание файла**

In [ ]:
s3.download_file('rag-documents-bucket', 'remote/path/file.txt', 'local_copy.txt')

## 3. Поиск по данным: векторные эмбеддинги

    Для RAG нужно преобразовать текст в векторы и искать похожие.

**1. Получение эмбеддингов через Yandex Embeddings API**

In [ ]:
import grpc

EMBEDDING_MODEL_URI = "emb://<folder-id>/text-search-query/latest"  # или text-search-doc

def get_embeddings(texts, iam_token, folder_id):
    channel = grpc.secure_channel('llm.api.cloud.yandex.net:443', grpc.ssl_channel_credentials())
    stub = text_embedding_service_pb2_grpc.TextEmbeddingServiceStub(channel)

    request = text_embedding_service_pb2.TextEmbeddingRequest(
        model_uri=EMBEDDING_MODEL_URI,
        texts=texts,
        folder_id=folder_id
    )

    metadata = [('authorization', f'Bearer {iam_token}')]
    response = stub.Embed(request, metadata=metadata)

    return [embedding.embedding for embedding in response.embeddings]

    Используйте text-search-query для запросов, text-search-doc для документов.

**2. Хранение и поиск эмбеддингов** <br>
Рекомендуется использовать векторную БД:
* FAISS (локально, для тестов)
* Qdrant / Milvus / PGVector (для продакшена)

Пример с FAISS:

In [ ]:
import faiss
import numpy as np

# Предположим, у вас есть список текстов и их эмбеддинги
texts = ["текст 1", "текст 2", ...]
embeddings = get_embeddings(texts, IAM_TOKEN, FOLDER_ID)

# Создаём индекс FAISS
dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings).astype('float32'))

# Поиск по запросу
query = "Какой у вас возврат товара?"
query_embedding = get_embeddings([query], IAM_TOKEN, FOLDER_ID)[0]
query_embedding = np.array([query_embedding]).astype('float32')

distances, indices = index.search(query_embedding, k=3)  # Топ-3 результата

for i in indices[0]:
    print(f"Релевантный фрагмент: {texts[i]}")

## 4. Интеграция с YandexGPT для RAG

Теперь передадим найденные фрагменты в YandexGPT как контекст.

**1. Вызов YandexGPT через API**

In [ ]:
from yandex.cloud.ai.foundation_models.v1 import completion_service_pb2_grpc, completion_service_pb2

GPT_MODEL_URI = "gpt://<folder-id>/yandexgpt/latest"

def ask_yandexgpt_with_context(iam_token, folder_id, context, question):
    channel = grpc.secure_channel('llm.api.cloud.yandex.net:443', grpc.ssl_channel_credentials())
    stub = completion_service_pb2_grpc.CompletionServiceStub(channel)

    prompt = f"""
    Контекст:
    {context}

    Вопрос: {question}
    Ответь максимально точно, основываясь только на контексте.
    """

    request = completion_service_pb2.CompletionRequest(
        model_uri=GPT_MODEL_URI,
        completion_options=completion_service_pb2.CompletionOptions(
            stream=False,
            temperature=0.3,
            max_tokens=1000
        ),
        messages=[
            completion_service_pb2.Message(
                role="system",
                text="Ты — помощник, отвечающий на вопросы на основе предоставленного контекста."
            ),
            completion_service_pb2.Message(
                role="user",
                text=prompt
            )
        ]
    )

    metadata = [('authorization', f'Bearer {iam_token}')]
    response = stub.Completion(request, metadata=metadata)

    return response.result.alternatives[0].message.text

**2. Полный RAG-пайплайн**

In [ ]:
def rag_pipeline(question, index, texts, iam_token, folder_id):
    # 1. Получить эмбеддинг вопроса
    query_embedding = get_embeddings([question], iam_token, folder_id)[0]
    query_embedding = np.array([query_embedding]).astype('float32')

    # 2. Найти топ-3 релевантных фрагмента
    distances, indices = index.search(query_embedding, k=3)
    context = "\n\n".join([texts[i] for i in indices[0]])

    # 3. Запрос к YandexGPT с контекстом
    answer = ask_yandexgpt_with_context(iam_token, folder_id, context, question)

    return answer

# Пример использования
question = "Как вернуть товар?"
answer = rag_pipeline(question, index, texts, IAM_TOKEN, FOLDER_ID)
print("Ответ:", answer)

## 5. Опционально: Автоматизация через Serverless Functions

Вы можете вынести RAG-логику в Yandex Cloud Functions:
* HTTP-триггер для API-запросов
* Фоновая функция для индексации новых документов
* Используйте runtime: python39 и подключите зависимости через requirements.txt

Пример requirements.txt:
>grpcio>=1.59.0<br>
>grpcio-tools>=1.59.0<br>
>boto3>=1.28.0<br>
>python-dotenv>=1.0.0<br>
>numpy>=1.24.0<br>
>faiss-cpu>=1.7.4<br>

## 6. Безопасность и лучшие практики

* Храните ключи в Secrets Manager или .env
* Используйте IAM-роли с минимальными привилегиями
* Кэшируйте IAM-токены (обновляйте каждые 11 часов)
* Логируйте запросы и ошибки
* Ограничьте длину контекста для YandexGPT (до 8K токенов)

## 7. Мониторинг и логирование

* Включите логирование в Cloud Logging
* Мониторьте использование квот через Billing → Usage
* Используйте Yandex Monitoring для алертов

## 8. Расширения

* Добавьте OCR (через Vision API) для сканированных PDF
* Подключите SpeechKit для голосовых запросов
* Используйте DataLens для визуализации статистики запросов
* Добавьте кэширование ответов (Redis или Object Storage)

**Пример структуры проекта**

rag-yandex/ <br>
├── .env <br>
├── main.py <br>
├── rag_pipeline.py <br>
├── storage.py <br>
├── embeddings.py <br>
├── gpt.py <br>
├── requirements.txt <br>
└── docs/ <br>
    └── (ваши PDF, TXT, DOCX) <br>

## P.S.
Документация Yandex Cloud AI: https://cloud.yandex.ru/docs/foundation-models